In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sklearn
import math
import joblib
from matplotlib import pyplot as plt

In [2]:
# Load the npz file
train_1 = np.load("data/train_data.npy")
train_2 = np.load("data/train_labels.npy")
train_df = DataFrame(train_1)
del train_1
train_lb_df = DataFrame(train_2)
del train_2

# For validation
val_1 = np.load("data/eval_data.npy")
val_2 = np.load("data/eval_labels.npy")
val_df = DataFrame(val_1)
del val_1
val_lb_df = DataFrame(val_2)
del val_2

# for unit_test

uni_df = train_df.iloc[100:,:]
uni_lb_df = train_lb_df.iloc[100:,:]

### To see the features importance
By random forest algo

In [3]:
# Load the forest
import joblib
forest = joblib.load( "Model/01_RF_training_350sqrt" )
ft_imp = pd.DataFrame(forest.feature_importances_)
ft_imp.index = train_df.columns
ft_imp = ft_imp.sort_values(0, ascending=False)
ft_imp["feature_name"] = ft_imp.index
ft_imp["importance"] = ft_imp.iloc[:,0]
ft_imp = ft_imp[ ["feature_name", "importance"] ]
for i in range(len(ft_imp)):
    print("%s %0.6f" % (str(ft_imp.iloc[i,0]), ft_imp.iloc[i,1]))
    #print(ft_imp.iloc[i,0])
ft_imp.info()

24 0.095185
49 0.057753
0 0.048706
18 0.023706
47 0.019150
21 0.018729
46 0.018422
45 0.017218
43 0.016490
6 0.016352
52 0.015682
41 0.015469
9 0.014890
48 0.014589
54 0.014339
22 0.013709
10 0.013667
39 0.013586
11 0.013571
51 0.013370
25 0.013327
38 0.013234
23 0.013207
20 0.013116
50 0.013089
63 0.013043
53 0.013013
37 0.012943
7 0.012797
28 0.012759
19 0.012742
8 0.012593
66 0.012492
13 0.012392
35 0.011987
4 0.011940
12 0.011888
26 0.011841
32 0.011759
29 0.011734
1 0.011686
44 0.011672
62 0.011596
42 0.011472
27 0.011305
67 0.011251
64 0.011158
30 0.011042
68 0.010927
33 0.010921
36 0.010892
34 0.010874
40 0.010805
69 0.010652
65 0.010623
31 0.010577
61 0.010491
14 0.009905
15 0.009512
5 0.009510
56 0.009018
16 0.008982
2 0.008759
17 0.008314
3 0.007885
58 0.005100
57 0.004243
59 0.001958
60 0.001336
55 0.001057
<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 24 to 55
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        -----

### Abandon the feature with < 0.005 importance

In [4]:
imp_notdrop_list = [ x for x in range(70) if x not in [55,57,59,60] ]
train_df = train_df[imp_notdrop_list]
val_df = val_df[imp_notdrop_list]
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174461 entries, 0 to 1174460
Data columns (total 66 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1174461 non-null  float64
 1   1       1174461 non-null  float64
 2   2       1174461 non-null  float64
 3   3       1174461 non-null  float64
 4   4       1174461 non-null  float64
 5   5       1174461 non-null  float64
 6   6       1174461 non-null  float64
 7   7       1174461 non-null  float64
 8   8       1174461 non-null  float64
 9   9       1174461 non-null  float64
 10  10      1174461 non-null  float64
 11  11      1174461 non-null  float64
 12  12      1174461 non-null  float64
 13  13      1174461 non-null  float64
 14  14      1174461 non-null  float64
 15  15      1174461 non-null  float64
 16  16      1174461 non-null  float64
 17  17      1174461 non-null  float64
 18  18      1174461 non-null  float64
 19  19      1174461 non-null  float64
 20  20      1174461 non-null

In [5]:
# To compare i and j
def Training_EnsembleModel(X, y, i=0, j=1, n_events=100000, name="" ):
    """
    import random
    start_idx = random.randrange(len(train_lb_df)-n_events)
    end_idx = start_idx + n_events
    
    print("training events start and end point : %f, %f" %(start_idx, end_idx))
    
    X = train_df.loc[((train_lb_df[0]==i)|(train_lb_df[0]==j)) & ((train_lb_df.index>=start_idx) & (train_lb_df.index<end_idx)),:]
    y = train_lb_df.loc[((train_lb_df[0]==i)|(train_lb_df[0]==j)) & ((train_lb_df.index>=start_idx) & (train_lb_df.index<end_idx)),0]
    
    print(X.shape)
    print(y.shape)
    """
    #X = train_df.loc[((train_lb_df[0]==i)|(train_lb_df[0]==j)) & (train_lb_df.index>1000000),:]
    #y = train_lb_df.loc[((train_lb_df[0]==i)|(train_lb_df[0]==j)) & (train_lb_df.index>1000000),0]
    
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import VotingClassifier
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.pipeline import make_pipeline
    from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
    
    lr = LogisticRegression(C=0.001)
    pipeline = make_pipeline(StandardScaler(), lr)
    
    forest = RandomForestClassifier( n_estimators=350, max_features=None, criterion='gini', random_state=22, n_jobs=-1 )
    
    tree = DecisionTreeClassifier( random_state=1, max_depth=12, max_features="sqrt", max_leaf_nodes=None )
    ABC = AdaBoostClassifier( base_estimator=tree, random_state=11, learning_rate=0.1, n_estimators=70 )
    
    """
    lr = LogisticRegression(C=lr_c)
    pipeline = make_pipeline(StandardScaler(), lr)
    
    forest = RandomForestClassifier( n_estimators=rf_nest, max_features=rf_maxf, criterion='gini', random_state=22, n_jobs=-1 )
    
    tree = DecisionTreeClassifier( random_state=1, max_depth=bdt_maxd, max_features=bdt_maxf, max_leaf_nodes=bdt_maxl )
    ABC = AdaBoostClassifier( base_estimator=tree, random_state=11, learning_rate=bdt_lr, n_estimators=bdt_nest )
    """
    model = VotingClassifier(estimators=[
            ('lr', pipeline), ('rf', forest), ('bdt', ABC)],
            voting='soft')
    model = model.fit( train_df.loc[((train_lb_df[0]==i)|(train_lb_df[0]==j)) & (train_lb_df.index>1000000),:], train_lb_df.loc[((train_lb_df[0]==i)|(train_lb_df[0]==j)) & (train_lb_df.index>1000000),0])
    joblib.dump( model, "01_final_Ensemble_training_afterCut_" + str(i) + "_" + str(j) + "_" + name )
    print("Accuracy : ", accuracy_score(val_lb_df.loc[(val_lb_df[0]==i)|(val_lb_df[0]==j),:],  model.predict(val_df.loc[(val_lb_df[0]==i)|(val_lb_df[0]==j),:])))

In [6]:
import random
n_events = 100000
start_idx = random.randrange(len(train_lb_df)-n_events)
end_idx = start_idx + n_events
Training_EnsembleModel(train_df.loc[((train_lb_df[0]==0)|(train_lb_df[0]==1)) & ((train_lb_df.index>=start_idx) & (train_lb_df.index<end_idx)),:],
                       train_lb_df.loc[((train_lb_df[0]==0)|(train_lb_df[0]==1)) & ((train_lb_df.index>=start_idx) & (train_lb_df.index<end_idx)),0],
                       0, 1)

Accuracy :  0.7416534001483438
